In [2]:
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install statsmodels
# !{sys.executable} -m pip install tabulate
# !{sys.executable} -m pip install httpimport

import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.iolib.summary2 import summary_col
from tabulate import tabulate

In [26]:
data_panel_diff = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data_election_crime_merged.csv')

C:\Users\mariu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
data_panel_diff = pd.concat([data_panel_diff, pd.get_dummies(data_panel_diff['party'], prefix='party')],axis=1)
data_panel_diff = pd.concat([data_panel_diff, pd.get_dummies(data_panel_diff['bl_kuerzel'], prefix='bl')],axis=1)

### Let's consider the standard measure of crime (against a party in a city)

In [28]:
data_panel_diff = data_panel_diff[data_panel_diff['cycle_1'] == 2014]
data_panel_diff = data_panel_diff.drop_duplicates(['city', 'party', 'plz', 'state', 'p_female_diff', 'crime_count_party'], ignore_index=True)
data_panel_diff

,Unnamed: 0,crime,city,law,date,background,suspects,party,state,plz,...,party_AfD,party_CDU,party_DIE LINKE,party_FDP,party_GRÜNE,party_SPD,bl_BB,bl_BW,bl_SN,bl_TH
0,54,Sachbeschadigung,Stuttgart,303 StGB,2019-02-16,Links,0.0,AfD,BW,70173.0,...,1,0,0,0,0,0,0,1,0,0
1,56,NaN,Stuttgart,NaN,NaN,NaN,NaN,CDU,BW,70173.0,...,0,1,0,0,0,0,0,0,0,0
2,57,NaN,Stuttgart,NaN,NaN,NaN,NaN,DIE LINKE,BW,70173.0,...,0,0,1,0,0,0,0,0,0,0
3,58,NaN,Stuttgart,NaN,NaN,NaN,NaN,FDP,BW,70173.0,...,0,0,0,1,0,0,0,0,0,0
4,59,NaN,Stuttgart,NaN,NaN,NaN,NaN,GRÜNE,BW,70173.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9242,20908,NaN,Pinnow,NaN,NaN,NaN,NaN,DIE LINKE,BB,16278.0,...,0,0,1,0,0,0,0,0,0,0
9243,20909,NaN,Pinnow,NaN,NaN,NaN,NaN,CDU,BB,16278.0,...,0,1,0,0,0,0,0,0,0,0
9244,20910,NaN,Pinnow,NaN,NaN,NaN,NaN,GRÜNE,BB,16278.0,...,0,0,0,0,1,0,0,0,0,0
9245,20914,NaN,Passow,NaN,NaN,NaN,NaN,CDU,BB,16306.0,...,0,1,0,0,0,0,0,0,0,0


In [29]:
data_panel_diff['p_female_diff'] = data_panel_diff['p_female_diff']*100

In [30]:
data_panel_diff['p_female_diff'].mean()

1.8368332446078404

In [31]:
data_panel_diff[['p_female_diff', 'city', 'crime_count_party']]

,p_female_diff,city,crime_count_party
0,6.666667,Stuttgart,13.0
1,-5.000000,Stuttgart,0.0
2,1.666667,Stuttgart,0.0
3,-1.666667,Stuttgart,2.0
4,0.000000,Stuttgart,1.0
...,...,...,...
9242,20.000000,Pinnow,0.0
9243,5.555556,Pinnow,0.0
9244,0.000000,Pinnow,0.0
9245,0.000000,Passow,0.0


In [32]:
print("Conditional means based on party (no restriction to wether a crime has happened or whether the party stood twice)")
data_panel_diff.groupby(['party'], as_index=False).mean()[['party', 'crime_count_party', 'p_female_diff']], len(data_panel_diff[~data_panel_diff['p_female_diff'].isna()])

Conditional means based on party (no restriction to wether a crime has happened or whether the party stood twice)


(       party  crime_count_party  p_female_diff
 0        AfD           0.329945       5.022525
 1        CDU           0.095969       1.393978
 2  DIE LINKE           0.000000       2.868497
 3        FDP           0.033910       1.560927
 4      GRÜNE           0.072058       3.091604
 5        SPD           0.156553       1.363334, 3924)

In [33]:
# Drop missing values
data_panel_diff.dropna(subset=['p_female_diff', 'crime_count_party'], inplace=True)
print("Conditional means based on party when they stood for office twice")
data_panel_diff.groupby(['party'], as_index=False).mean()[['party', 'crime_count_party', 'p_female_diff']], len(data_panel_diff[~data_panel_diff['p_female_diff'].isna()])

Conditional means based on party when they stood for office twice


(       party  crime_count_party  p_female_diff
 0        AfD           4.981481       5.022525
 1        CDU           0.126549       1.393978
 2  DIE LINKE           0.000000       2.868497
 3        FDP           0.107612       1.560927
 4      GRÜNE           0.203125       3.091604
 5        SPD           0.259334       1.363334, 3924)

In [48]:
data_panel_diff['log_crime_count_party'] = np.log(data_panel_diff['crime_count_party']+1)
data_panel_diff

,Unnamed: 0,crime,city,law,date,background,suspects,party,state,plz,...,party_CDU,party_DIE LINKE,party_FDP,party_GRÜNE,party_SPD,bl_BB,bl_BW,bl_SN,bl_TH,log_crime_count_party
0,54,Sachbeschadigung,Stuttgart,303 StGB,2019-02-16,Links,0.0,AfD,BW,70173.0,...,0,0,0,0,0,0,1,0,0,2.639057
1,56,NaN,Stuttgart,NaN,NaN,NaN,NaN,CDU,BW,70173.0,...,1,0,0,0,0,0,0,0,0,0.000000
2,57,NaN,Stuttgart,NaN,NaN,NaN,NaN,DIE LINKE,BW,70173.0,...,0,1,0,0,0,0,0,0,0,0.000000
3,58,NaN,Stuttgart,NaN,NaN,NaN,NaN,FDP,BW,70173.0,...,0,0,1,0,0,0,0,0,0,1.098612
4,59,NaN,Stuttgart,NaN,NaN,NaN,NaN,GRÜNE,BW,70173.0,...,0,0,0,1,0,0,0,0,0,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9240,20905,NaN,Mark Landin,NaN,NaN,NaN,NaN,SPD,BB,16278.0,...,0,0,0,0,1,0,0,0,0,0.000000
9242,20908,NaN,Pinnow,NaN,NaN,NaN,NaN,DIE LINKE,BB,16278.0,...,0,1,0,0,0,0,0,0,0,0.000000
9243,20909,NaN,Pinnow,NaN,NaN,NaN,NaN,CDU,BB,16278.0,...,1,0,0,0,0,0,0,0,0,0.000000
9244,20910,NaN,Pinnow,NaN,NaN,NaN,NaN,GRÜNE,BB,16278.0,...,0,0,0,1,0,0,0,0,0,0.000000


In [49]:
X = data_panel_diff['log_crime_count_party']
y = data_panel_diff['p_female_diff']
X = sm.add_constant(X)
model1 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     3.396
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.317
Time:                        00:43:15   Log-Likelihood:                -16852.
No. Observations:                3924   AIC:                         3.371e+04
Df Residuals:                    3922   BIC:                         3.372e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.7536      0.268      6.548      0.000       1.229       2.279
log_crime_count_party     1.1895      0.646      1.843      0.065      -0.076       2.455
==============================================================================
Omnibus:                      626.207   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5829.555
Skew:                           0.470   Prob(JB):                         0.00
Kurtosis:                       8.897   Cond. No.                         3.16
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [50]:
model2 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party']})
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.288
Date:                Wed, 05 Jan 2022   Prob (F-statistic):             0.0698
Time:                        00:44:46   Log-Likelihood:                -16852.
No. Observations:                3924   AIC:                         3.371e+04
Df Residuals:                    3922   BIC:                         3.372e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.7536      0.317      5.533      0.000       1.132       2.375
log_crime_count_party     1.1895      0.517      2.300      0.021       0.176       2.203
==============================================================================
Omnibus:                      626.207   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5829.555
Skew:                           0.470   Prob(JB):                         0.00
Kurtosis:                       8.897   Cond. No.                         3.16
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [51]:
model3 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['state']})
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     12.96
Date:                Wed, 05 Jan 2022   Prob (F-statistic):             0.0368
Time:                        00:45:17   Log-Likelihood:                -16852.
No. Observations:                3924   AIC:                         3.371e+04
Df Residuals:                    3922   BIC:                         3.372e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.7536      0.694      2.528      0.011       0.394       3.113
log_crime_count_party     1.1895      0.330      3.600      0.000       0.542       1.837
==============================================================================
Omnibus:                      626.207   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5829.555
Skew:                           0.470   Prob(JB):                         0.00
Kurtosis:                       8.897   Cond. No.                         3.16
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [52]:
print(summary_col([model1,model2, model3]))
# A increase of crime against a party by 1% is associated with an 1.2% higher share of females in the following period. 


                      p_female_diff I p_female_diff II p_female_diff III
------------------------------------------------------------------------
const                 1.7536          1.7536           1.7536           
                      (0.2678)        (0.3169)         (0.6937)         
log_crime_count_party 1.1895          1.1895           1.1895           
                      (0.6455)        (0.5173)         (0.3304)         
Standard errors in parentheses.


In [77]:
%run estout_func.py

In [54]:
X2 = data_panel_diff[["log_crime_count_party", "party_GRÜNE", "party_SPD", "party_CDU", "party_FDP", "party_AfD", 'bl_BB', 'bl_SN', 'bl_TH']]
X2 = sm.add_constant(X2)
model4 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model4.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.314
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.457
Time:                        00:49:00   Log-Likelihood:                -16847.
No. Observations:                3924   AIC:                         3.371e+04
Df Residuals:                    3914   BIC:                         3.378e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.8685    1.8e-14   1.59e+14      0.000       2.868       2.868
log_crime_count_party     1.8077      1.791      1.010      0.313      -1.702       5.317
party_GRÜNE               0.1165      0.090      1.301      0.193      -0.059       0.292
party_SPD                -1.6454      0.132    -12.482      0.000      -1.904      -1.387
party_CDU                -1.5281      0.057    -26.728      0.000      -1.640      -1.416
party_FDP                -1.3838      0.067    -20.719      0.000      -1.515      -1.253
party_AfD                 1.3263      0.720      1.843      0.065      -0.085       2.737
bl_BB                    -0.3010      2.111     -0.143      0.887      -4.438       3.836
bl_SN                    -4.6722      4.321     -1.081      0.280     -13.141       3.796
bl_TH                    -9.5175      7.442     -1.279      0.201     -24.104       5.069
==============================================================================
Omnibus:                      600.927   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5769.194
Skew:                           0.421   Prob(JB):                         0.00
Kurtosis:                       8.880   Cond. No.                         32.4
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [39]:
model5 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party']})
model5.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 4
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     25.95
Date:                Wed, 05 Jan 2022   Prob (F-statistic):            0.00153
Time:                        00:34:52   Log-Likelihood:                -16848.
No. Observations:                3924   AIC:                         3.372e+04
Df Residuals:                    3914   BIC:                         3.378e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.8685   2.76e-15   1.04e+15      0.000       2.868       2.868
crime_count_party     0.0992      0.126      0.789      0.430      -0.147       0.346
party_GRÜNE           0.2227      0.056      3.979      0.000       0.113       0.332
party_SPD            -1.5172      0.039    -38.554      0.000      -1.594      -1.440
party_CDU            -1.4626      0.012   -124.570      0.000      -1.486      -1.440
party_FDP            -1.3192      0.027    -48.689      0.000      -1.372      -1.266
party_AfD             1.8288      0.663      2.757      0.006       0.529       3.129
bl_BB                 1.4093      4.007      0.352      0.725      -6.444       9.263
bl_SN                -2.4580      1.750     -1.404      0.160      -5.888       0.972
bl_TH                -7.2601      4.049     -1.793      0.073     -15.195       0.675
==============================================================================
Omnibus:                      598.021   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5764.197
Skew:                           0.415   Prob(JB):                         0.00
Kurtosis:                       8.879   Cond. No.                         60.1
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [40]:
model6 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['state']})
model6.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 3
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6669
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.626
Time:                        00:34:58   Log-Likelihood:                -16848.
No. Observations:                3924   AIC:                         3.372e+04
Df Residuals:                    3914   BIC:                         3.378e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.8685      0.337      8.518      0.000       2.208       3.529
crime_count_party     0.0992      0.057      1.749      0.080      -0.012       0.210
party_GRÜNE           0.2227      1.299      0.171      0.864      -2.324       2.770
party_SPD            -1.5172      0.273     -5.561      0.000      -2.052      -0.982
party_CDU            -1.4626      0.271     -5.398      0.000      -1.994      -0.931
party_FDP            -1.3192      0.748     -1.764      0.078      -2.785       0.147
party_AfD             1.8288      3.120      0.586      0.558      -4.287       7.945
bl_BB                 1.4093      1.153      1.223      0.221      -0.850       3.668
bl_SN                -2.4580      1.027     -2.394      0.017      -4.471      -0.445
bl_TH                -7.2601      0.768     -9.458      0.000      -8.765      -5.756
==============================================================================
Omnibus:                      598.021   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5764.197
Skew:                           0.415   Prob(JB):                         0.00
Kurtosis:                       8.879   Cond. No.                         60.1
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [89]:
estout_ols(modellist=[model4, model5, model6], modellist_str = ["model4","model5","model6"], y='Share Female Candidates', caption="Basic Regression", label="Basic Regression", list_regression_sets=[X3,X3,X3], p_values=True)

\begin{table}[htbp] \caption{Basic Regression \label{Basic Regression}}
\resizebox{0.9\textwidth}{!}{ \centering
\begin{tabular}{lccc} \hline
  & (I) & (II) & (III) \\ 
Dependent Variable & \multicolumn{3}{c}{Share Female Candidates} \\ \hline \vspace{4pt} 
& \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize}  \\ 
bl_BB & -1.5629 & -1.5629 & -1.5629 \\ 
 \vspace{4pt} & \begin{footnotesize}(5.9921) \end{footnotesize} & \begin{footnotesize}(4.5956) \end{footnotesize} & \begin{footnotesize}(2.4601) \end{footnotesize} \\ 
bl_SN & 3.2762*** & 3.2762* & 3.2762** \\ 
 \vspace{4pt} & \begin{footnotesize}(0.9389) \end{footnotesize} & \begin{footnotesize}(1.829) \end{footnotesize} & \begin{footnotesize}(1.6403) \end{footnotesize} \\ 
bl_TH & -1.1523 & -1.1523 & -1.1523 \\ 
 \vspace{4pt} & \begin{footnotesize}(4.4142) \end{footnotesize} & \begin{footnotesize}(2.9738) \end{footnotesize} & \begin{footnotesize}(0.9002) \end{footno

In [41]:
data_panel_diff_balanced_noafd = data_panel_diff[data_panel_diff['party_AfD'] != 1]
X4 = data_panel_diff_balanced_noafd[["crime_count_party", "party_SPD", "party_FDP", "party_CDU", "party_GRÜNE", 'bl_BB', 'bl_SN', 'bl_TH']]
y3 = data_panel_diff_balanced_noafd["p_female_diff"]
X4 = sm.add_constant(X4)
model9 = sm.OLS(y3, X4).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff_balanced_noafd['party_lr']})
model9.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.056
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.491
Time:                        00:35:10   Log-Likelihood:                -16560.
No. Observations:                3870   AIC:                         3.314e+04
Df Residuals:                    3861   BIC:                         3.319e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.8685   1.43e-14      2e+14      0.000       2.868       2.868
crime_count_party     0.5439      0.255      2.133      0.033       0.044       1.044
party_SPD            -1.6132      0.014   -114.563      0.000      -1.641      -1.586
party_FDP            -1.3618      0.002   -643.657      0.000      -1.366      -1.358
party_CDU            -1.4981      0.001  -1940.067      0.000      -1.500      -1.497
party_GRÜNE           0.1572      0.046      3.412      0.001       0.067       0.247
bl_BB                 2.1257      3.957      0.537      0.591      -5.629       9.881
bl_SN                -5.8957      3.349     -1.761      0.078     -12.459       0.667
bl_TH               -10.0528      7.754     -1.297      0.195     -25.250       5.144
==============================================================================
Omnibus:                      575.875   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5439.089
Skew:                           0.401   Prob(JB):                         0.00
Kurtosis:                       8.752   Cond. No.                         34.2
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [46]:
data_panel_diff_balanced_noafd = data_panel_diff[data_panel_diff['party_SPD'] != 1]
X5 = data_panel_diff_balanced_noafd[["crime_count_party", "party_GRÜNE", "party_FDP", "party_CDU", "party_AfD", 'bl_BB', 'bl_SN', 'bl_TH']]
y3 = data_panel_diff_balanced_noafd["p_female_diff"]
X5 = sm.add_constant(X5)
model10 = sm.OLS(y3, X5).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff_balanced_noafd['party_lr']})
model10.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3606
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.656
Time:                        00:39:12   Log-Likelihood:                -12680.
No. Observations:                2933   AIC:                         2.538e+04
Df Residuals:                    2924   BIC:                         2.543e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.8685   2.43e-14   1.18e+14      0.000       2.868       2.868
crime_count_party     0.0388      0.088      0.443      0.658      -0.133       0.210
party_GRÜNE           0.2605      0.046      5.621      0.000       0.170       0.351
party_FDP            -1.2982      0.015    -86.033      0.000      -1.328      -1.269
party_CDU            -1.4642      0.002   -798.105      0.000      -1.468      -1.461
party_AfD             2.2618      0.077     29.406      0.000       2.111       2.413
bl_BB                -1.8448      3.541     -0.521      0.602      -8.785       5.095
bl_SN                -1.4797      2.257     -0.656      0.512      -5.903       2.943
bl_TH                -3.7363      0.153    -24.429      0.000      -4.036      -3.437
==============================================================================
Omnibus:                      463.135   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4815.138
Skew:                           0.410   Prob(JB):                         0.00
Kurtosis:                       9.223   Cond. No.                         62.4
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [43]:
data_panel_diff2 = data_panel_diff.drop_duplicates(['city', 'plz', 'state', 'p_female_city_diff', 'crime_count_city'], ignore_index=True)
data_panel_diff2.dropna(subset=['p_female_city_diff', 'crime_count_city'], inplace=True)

X2 = data_panel_diff2["crime_count_city"]
y = data_panel_diff2["p_female_city_diff"]
X2 = sm.add_constant(X2)
model4 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff2['party_lr']})
model4.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     p_female_city_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     3.577
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.310
Time:                        00:35:46   Log-Likelihood:                 1147.0
No. Observations:                1668   AIC:                            -2290.
Df Residuals:                    1666   BIC:                            -2279.
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0033      0.006      0.598      0.550      -0.008       0.014
crime_count_city     0.0002      0.000      1.891      0.059    -7.9e-06       0.000
==============================================================================
Omnibus:                      289.411   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2736.752
Skew:                          -0.515   Prob(JB):                         0.00
Kurtosis:                       9.190   Cond. No.                         5.97
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [44]:
X2 = data_panel_diff2["crime_count_city"]
y = data_panel_diff2["p_female_city_diff"]
X2 = sm.add_constant(X2)
model5 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff2['party']})
model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     p_female_city_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     1.442
Date:                Wed, 05 Jan 2022   Prob (F-statistic):              0.284
Time:                        00:35:55   Log-Likelihood:                 1147.0
No. Observations:                1668   AIC:                            -2290.
Df Residuals:                    1666   BIC:                            -2279.
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0033      0.004      0.813      0.416      -0.005       0.011
crime_count_city     0.0002      0.000      1.201      0.230      -0.000       0.001
==============================================================================
Omnibus:                      289.411   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2736.752
Skew:                          -0.515   Prob(JB):                         0.00
Kurtosis:                       9.190   Cond. No.                         5.97
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [45]:
data_panel_lr = data_panel_diff.drop_duplicates(['city', 'party', 'plz', 'state', 'p_female_party_lr', 'crime_count_party_lr'], ignore_index=True)
data_panel_lr.dropna(subset=['p_female_party_lr', 'crime_count_party_lr'], inplace=True)

X2 = data_panel_lr["crime_count_party_lr"]
y = data_panel_lr["p_female_party_lr"]
X2 = sm.add_constant(X2)
model6 = sm.OLS(y, X2).fit(cov_type='HC3')
model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      p_female_party_lr   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     9.118
Date:                Wed, 05 Jan 2022   Prob (F-statistic):            0.00381
Time:                        00:35:58   Log-Likelihood:                 28.816
No. Observations:                  58   AIC:                            -53.63
Df Residuals:                      56   BIC:                            -49.51
Df Model:                           1                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.2861      0.022     12.900      0.000       0.243       0.330
crime_count_party_lr    -0.0022      0.001     -3.020      0.003      -0.004      -0.001
==============================================================================
Omnibus:                        1.418   Durbin-Watson:                   2.173
Prob(Omnibus):                  0.492   Jarque-Bera (JB):                1.349
Skew:                          -0.254   Prob(JB):                        0.509
Kurtosis:                       2.452   Cond. No.                         16.7
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

---